In [ ]:
!pip install faiss-cpu transformers torch huggingface_hub
from IPython.display import clear_output
clear_output()

In [ ]:
from huggingface_hub import login

login()

In [ ]:
import json
import faiss
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModel
import torch
from tqdm import tqdm

In [ ]:
with open("/kaggle/input/anime-dataset/result.json", "r", encoding="utf-8") as f:
    anime_data = json.load(f)

In [ ]:
# Load embedding model (same as in the Colab notebook)
embedding_model = AutoModel.from_pretrained("BAAI/bge-small-en-v1.5")
embedding_tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-small-en-v1.5")

# Load Mistral model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1", torch_dtype=torch.float16, device_map="auto")

In [ ]:
# Generate embeddings
def get_embedding(text):
    inputs = embedding_tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = embedding_model(**inputs)
    sentence_embedding = outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()
    return sentence_embedding

# Build Faiss Index
def build_faiss_index(anime_data):
    dimension = 384  
    index = faiss.IndexFlatL2(dimension)
    embeddings = np.array([get_embedding(text) for text in tqdm(anime_data)], dtype=np.float32)
    index.add(embeddings)
    return index, embeddings

def search_faiss(query, anime_data, index, top_k=3):
    query_embedding = np.array([get_embedding(query)], dtype=np.float32)
    _, indices = index.search(query_embedding, top_k)
    return [anime_data[i] for i in indices[0]]

def generate_response(context, query):
    prompt = f"You are an anime expert. Answer based on the following data:\n{context}\n\nUser query: {query}\nAnswer: "
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    output = model.generate(**inputs, max_new_tokens=300, do_sample=True)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response[response.index("Answer:"):]

def chatbot(anime_data, index):
    print("Welcome to the Anime Chatbot! Type 'exit' to stop.")
    while True:
        user_input = input("Ask anything about anime: ").strip()
        if user_input.lower() == "exit":
            print("Goodbye!")
            break
        retrieved_data = search_faiss(user_input, anime_data, index)
        context = "\n".join(retrieved_data)
        response = generate_response(context, user_input)
        print(f"\n{response}\n")


In [ ]:
try:
    faiss.read_index(index, "anime_faiss.index")
except:
    index, _ = build_faiss_index(anime_data)
    faiss.write_index(index, "anime_faiss.index")

In [ ]:
chatbot(anime_data, index)